# YOLO小目标检测优化方案（4050 Laptop显存优化版）

本Notebook用于处理label studio导出的YOLO格式数据，通过缩小图像增强模型对小目标的敏感度。
**关键优化**：YOLO标签采用归一化坐标，缩放图像后无需修改标签文件，直接复制即可，大幅节省计算资源。

In [8]:
# 步骤1：安装依赖（仅第一次运行需要）
# !pip install ultralytics==8.2.0 pillow tqdm psutil

In [9]:
# ⚠️ 必须在任何导入之前执行此单元格
import os
import sys

# 修复OpenMP冲突（解决libiomp5md.dll重复初始化问题）
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
# os.environ['OMP_NUM_THREADS'] = '1'  # 限制OpenMP线程数

# 可选：完全禁用OpenMP（更激进但更安全）
# os.environ['MKL_THREADING_LAYER'] = 'GNU'

print("✅ OpenMP环境变量已设置")
print(f"🔧 KMP_DUPLICATE_LIB_OK = {os.getenv('KMP_DUPLICATE_LIB_OK')}")
print(f"🔧 OMP_NUM_THREADS = {os.getenv('OMP_NUM_THREADS')}")

✅ OpenMP环境变量已设置
🔧 KMP_DUPLICATE_LIB_OK = TRUE
🔧 OMP_NUM_THREADS = None


In [10]:
# 步骤2：导入库
import os
from pathlib import Path
from PIL import Image
import shutil
from tqdm.notebook import tqdm
import yaml
import torch
import psutil
import gc

# 清理GPU缓存
torch.cuda.empty_cache()

In [11]:
# 步骤3：配置参数（按需修改）
INPUT_FOLDER = "data/1210-403"  # 原始数据集文件夹
SCALE_FACTOR = 4           # 缩小倍数（默认1/4）
BATCH_SIZE = 16             # 根据RTX 4050 Laptop 6GB显存设置为4最安全
IMAGE_SIZE = 640           # 训练图像尺寸

In [12]:
# 步骤4：内存监控函数（用于实时观察资源使用）
def print_memory_usage():
    """打印当前内存和显存使用情况"""
    # GPU显存
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        max_allocated = torch.cuda.max_memory_allocated() / 1024**3
        print(
            f"📊 GPU显存: {allocated:.2f}GB(已用) / {reserved:.2f}GB(保留) / {max_allocated:.2f}GB(峰值)")

    # 系统内存
    memory = psutil.virtual_memory()
    print(
        f"💻 系统内存: {memory.used / 1024**3:.2f}GB(已用) / {memory.total / 1024**3:.2f}GB(总计) / {memory.percent}%")
    print("-" * 60)


print_memory_usage()

📊 GPU显存: 0.00GB(已用) / 0.00GB(保留) / 0.00GB(峰值)
💻 系统内存: 11.68GB(已用) / 15.81GB(总计) / 73.9%
------------------------------------------------------------


In [13]:
# 步骤5：核心函数 - 生成缩小数据集并自动划分 train/val
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random


def create_downsampled_dataset(
        input_folder: str,
        scale_factor: int = 4,
        val_split: float = 0.2,
        seed: int = 42):
    """
    创建缩小版数据集，并自动划分 train/val
    - 图像按比例缩小
    - YOLO标签直接复制（归一化坐标无需转换）
    - val_split: 验证集比例，默认 20%
    """
    input_path = Path(input_folder)
    if not input_path.exists():
        raise FileNotFoundError(f"输入文件夹不存在: {input_folder}")

    # 固定随机种子，保证可复现
    random.seed(seed)

    # 输出目录结构：{原文件夹名}-det
    folder_name = input_path.name
    output_path = input_path.parent / f"{folder_name}-det"
    if output_path.exists():          # 清空旧数据
        shutil.rmtree(output_path)

    # 新建目录
    def make_dirs(*sub):
        (output_path / Path(*sub)).mkdir(parents=True, exist_ok=True)

    make_dirs("images", "train")
    make_dirs("images", "val")
    make_dirs("labels", "train")
    make_dirs("labels", "val")

    # 复制配置文件
    for config_file in ["classes.txt", "notes.json"]:
        src = input_path / config_file
        if src.exists():
            shutil.copy2(src, output_path / config_file)
            print(f"✅ 已复制 {config_file}")

    # 收集所有图片（支持嵌套子目录）
    img_suffix = {".jpg", ".jpeg", ".png", ".bmp"}
    image_files = []
    for suf in img_suffix:
        image_files.extend(input_path.rglob(f"*{suf}"))
        image_files.extend(input_path.rglob(f"*{suf.upper()}"))

    if not image_files:
        raise ValueError("未找到任何图像文件")

    # 去重并排序，方便划分
    image_files = sorted(set(image_files), key=lambda x: x.name)
    print(f"🖼️ 共发现 {len(image_files)} 张图像，开始处理...")

    # 先全部缩小+复制标签到临时目录
    tmp_img_dir = output_path / "images_tmp"
    tmp_lbl_dir = output_path / "labels_tmp"
    tmp_img_dir.mkdir(exist_ok=True)
    tmp_lbl_dir.mkdir(exist_ok=True)

    success_list = []          # 记录成功的文件名（无后缀）
    for img_path in tqdm(image_files, desc="缩小+复制标签"):
        try:
            with Image.open(img_path) as img:
                new_size = (img.width // scale_factor,
                            img.height // scale_factor)
                img_resized = img.resize(new_size, Image.Resampling.LANCZOS)
                tmp_img = tmp_img_dir / img_path.name
                img_resized.save(tmp_img, quality=95, optimize=True)

            # 复制标签
            lbl_path = img_path.parents[1] / "labels" / f"{img_path.stem}.txt" \
                if img_path.parent.name == "images" \
                else img_path.with_suffix(".txt").parent.parent / "labels" / f"{img_path.stem}.txt"
            if not lbl_path.exists():
                lbl_path = img_path.with_suffix(".txt")   # 兜底：同目录
            if lbl_path.exists():
                shutil.copy2(lbl_path, tmp_lbl_dir / f"{img_path.stem}.txt")

            success_list.append(img_path.stem)
        except Exception as e:
            print(f"❌ 处理失败 {img_path.name}: {e}")
            continue

    print(f"✅ 成功处理 {len(success_list)} 张图像，开始划分 train/val...")

    # 划分
    train_files, val_files = train_test_split(
        success_list, test_size=val_split, random_state=seed)

    # 移动文件到最终目录
    def move_set(file_stems, split):
        for stem in file_stems:
            shutil.move(tmp_img_dir / f"{stem}.jpg",
                        output_path / "images" / split / f"{stem}.jpg")
            lbl = tmp_lbl_dir / f"{stem}.txt"
            if lbl.exists():
                shutil.move(lbl,
                            output_path / "labels" / split / f"{stem}.txt")

    move_set(train_files, "train")
    move_set(val_files, "val")

    # 清理临时目录
    shutil.rmtree(tmp_img_dir)
    shutil.rmtree(tmp_lbl_dir)

    print(f"✅ 划分完成：train={len(train_files)}，val={len(val_files)}")
    return output_path, len(success_list)

In [14]:
# 步骤6：执行
print(f"🚀 开始处理数据集: {INPUT_FOLDER}")
print(f"📏 缩小因子: 1/{SCALE_FACTOR}")
print("=" * 60)

output_dataset_path, total = create_downsampled_dataset(
    INPUT_FOLDER, scale_factor=SCALE_FACTOR, val_split=0.2)

print(f"\n🎯 新数据集路径: {output_dataset_path}")

🚀 开始处理数据集: data/1210-403
📏 缩小因子: 1/4
✅ 已复制 classes.txt
✅ 已复制 notes.json
🖼️ 共发现 403 张图像，开始处理...


缩小+复制标签: 100%|██████████| 403/403 [01:18<00:00,  5.14it/s]


✅ 成功处理 403 张图像，开始划分 train/val...
✅ 划分完成：train=322，val=81

🎯 新数据集路径: data\1210-403-det


In [15]:
# 步骤7：创建YOLO配置文件
def create_yolo_config(dataset_path: Path):
    """创建YOLO训练用的yaml配置文件"""
    # 读取类别
    classes_file = dataset_path / "classes.txt"
    class_names = {}
    if classes_file.exists():
        with open(classes_file, 'r', encoding='utf-8') as f:
            classes = [line.strip() for line in f if line.strip()]
            class_names = {i: name for i, name in enumerate(classes)}

    # 单类模式（忽略分类，专注检测）
    # 将所有类别合并为"target"
    config = {
        'path': str(dataset_path),
        'train': 'images',
        'val': 'images',  # 注意：实际项目中应单独划分验证集
        'nc': 1,  # 强制单类
        'names': {0: 'target'}  # 所有目标归为"target"类
    }

    config_path = dataset_path / "dataset.yaml"
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

    print(f"📄 配置文件已创建: {config_path}")
    print(f"📋 类别配置: {config['names']}")
    return config_path


config_path = create_yolo_config(output_dataset_path)

📄 配置文件已创建: data\1210-403-det\dataset.yaml
📋 类别配置: {0: 'target'}


In [16]:
# 步骤8：配置YOLO训练参数（4050 Laptop专用优化）
def get_train_config(config_path: str):
    """获取针对RTX 4050 Laptop优化的训练配置"""
    return {
        # 基础配置
        'data': config_path,
        'epochs': 100,          # 适中轮数
        'imgsz': IMAGE_SIZE,   # 图像尺寸
        'batch': BATCH_SIZE,   # 批大小（6GB显存安全值）

        # 设备配置
        'device': 0,           # GPU编号
        'workers': 1,          # 数据加载线程（减少内存占用）

        # 显存优化（关键）
        'amp': True,           # 混合精度训练（必备）
        'cache': 'disk',        # 磁盘缓存（节省内存）
        # 'fraction': 0.95,       # 使用95%显存

        # 模型配置
        'model': 'yolov8s.pt',  # nano模型（最小显存占用）
        # 'single_cls': True,    # 单类模式（忽略分类）
        'pretrained': True,    # 使用预训练权重

        # 优化器
        'optimizer': 'AdamW',  # AdamW收敛更快
        'lr0': 0.0001,                # 0.0005比较保守
        'cos_lr': True,             # 新增：余弦退火调度
        'warmup_epochs': 3,         # 新增：预热轮数
        'weight_decay': 0.0005,     # 新增：权重衰减
        # 'dropout': 0.2,            # 新增：轻度Dropout防过拟合
        'conf': 0.05,

        # 早停机制
        'patience': 10,        # 10轮无改善则停止

        # 数据增强强化
        'mosaic': 0.5,
        'copy_paste': 0.1,
        'mixup': 0.0,
        'degrees': 5,
        'perspective': 0.0,

        # Loss权重（专注定位）
        # 'box': 15,            # 提升框回归loss权重
        # 'cls': 0.5,            # 降低分类loss权重
        # 'dfl': 1.5,

        # 其他
        'save': True,
        'verbose': True,
        'project': f'{output_dataset_path.parent}/yolo_runs',
        'name': f'{output_dataset_path.name}_train'
    }


train_params = get_train_config(str(config_path))

print("🏋️ YOLO训练配置:")
for k, v in train_params.items():
    print(f"  {k}: {v}")
print("\n⚠️  重要提示:")
print("  - 每轮训练后自动清理显存")
print("  - 使用混合精度减少50%显存占用")
print("  - 单类模式忽略分类，专注框选精度")
print("=" * 60)
print_memory_usage()

🏋️ YOLO训练配置:
  data: data\1210-403-det\dataset.yaml
  epochs: 100
  imgsz: 640
  batch: 16
  device: 0
  workers: 1
  amp: True
  cache: disk
  model: yolov8s.pt
  pretrained: True
  optimizer: AdamW
  lr0: 0.0001
  cos_lr: True
  warmup_epochs: 3
  weight_decay: 0.0005
  conf: 0.05
  patience: 10
  mosaic: 0.5
  copy_paste: 0.1
  mixup: 0.0
  degrees: 5
  perspective: 0.0
  save: True
  verbose: True
  project: data/yolo_runs
  name: 1210-403-det_train

⚠️  重要提示:
  - 每轮训练后自动清理显存
  - 使用混合精度减少50%显存占用
  - 单类模式忽略分类，专注框选精度
📊 GPU显存: 0.00GB(已用) / 0.00GB(保留) / 0.00GB(峰值)
💻 系统内存: 11.98GB(已用) / 15.81GB(总计) / 75.8%
------------------------------------------------------------


In [17]:
# 步骤9：开始训练（核心步骤）
from ultralytics import YOLO

# 再次清理显存
torch.cuda.empty_cache()
print("🚀 初始化YOLO模型...")

# 加载模型（首次运行会自动下载权重）
model = YOLO(train_params['model'])
print(f"✅ 模型加载完成: {train_params['model']}")
print_memory_usage()

🚀 初始化YOLO模型...
✅ 模型加载完成: yolov8s.pt
📊 GPU显存: 0.00GB(已用) / 0.00GB(保留) / 0.00GB(峰值)
💻 系统内存: 12.05GB(已用) / 15.81GB(总计) / 76.2%
------------------------------------------------------------


In [18]:
# 步骤10：执行训练（带异常处理和内存监控）
try:
    print(f"\n🎯 开始训练! 目标: 提升小目标检测敏感度")
    print(f"📊 训练轮数: {train_params['epochs']} | 批大小: {train_params['batch']}")
    print("=" * 60)

    # 训练模型
    results = model.train(
        **{k: v for k, v in train_params.items() if k != 'model'})

    print("\n✅ 训练完成!")
    print(f"📁 模型保存路径: {train_params['project']}/{train_params['name']}")

except torch.cuda.OutOfMemoryError as e:
    print("\n❌ 显存不足错误!")
    print("💡 解决方案:")
    print("   1. 减小 BATCH_SIZE (当前: {})".format(BATCH_SIZE))
    print("   2. 减小 IMAGE_SIZE (当前: {})".format(IMAGE_SIZE))
    print("   3. 增大 SCALE_FACTOR (当前: {})".format(SCALE_FACTOR))
    print("   4. 重启Kernel后重试")

except Exception as e:
    print(f"\n❌ 训练出错: {type(e).__name__}")
    print(f"📄 错误信息: {str(e)}")

finally:
    # 最终清理
    torch.cuda.empty_cache()
    print("\n🧹 显存清理完成")
    print_memory_usage()


🎯 开始训练! 目标: 提升小目标检测敏感度
📊 训练轮数: 100 | 批大小: 16
New https://pypi.org/project/ultralytics/8.3.238 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.154  Python-3.11.7 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.05, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data\1210-403-det\dataset.yaml, degrees=5, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.5, multi_sca

train: Scanning D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\labels\train... 403 images, 0 backgrounds, 397 corrupt: 100%|██████████| 403/403 [00:00<00:00, 2378.80it/s]

train: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\images\train\FILE0001.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\images\train\FILE0002.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\images\train\FILE0003.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\images\train\FILE0004.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\images\train\FILE0006.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: D:\Ma_Siyuan\Perso

train: New cache created: D:\Ma_Siyuan\Personal\Code\FlyMorpher\data\1210-403-det\labels\train.cache


train: Caching images (0.0GB Disk): 100%|██████████| 6/6 [00:00<00:00, 630.71it/s]


...

     22/100      1.75G      1.393      1.221      1.372          8        640: 100%|██████████| 1/1 [00:00<00:00,  9.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 17.51it/s]

                   all          6          6      0.214        0.5       0.22     0.0334
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 12, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



22 epochs completed in 0.004 hours.
Optimizer stripped from data\yolo_runs\1210-403-det_train2\weights\last.pt, 22.5MB
Optimizer stripped from data\yolo_runs\1210-403-det_train2\weights\best.pt, 22.5MB

Validating data\yolo_runs\1210-403-det_train2\weights\best.pt...
Ultralytics 8.3.154  Python-3.11.7 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 18.82it/s]


                   all          6          6      0.188        0.5      0.203      0.108
Speed: 0.2ms preprocess, 3.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to data\yolo_runs\1210-403-det_train2

✅ 训练完成!
📁 模型保存路径: data/yolo_runs/1210-403-det_train

🧹 显存清理完成
📊 GPU显存: 0.25GB(已用) / 0.68GB(保留) / 1.69GB(峰值)
💻 系统内存: 14.73GB(已用) / 15.81GB(总计) / 93.1%
------------------------------------------------------------


In [19]:
# 最终内存状态
print("\n📊 最终资源状态:")
print_memory_usage()


📊 最终资源状态:
📊 GPU显存: 0.25GB(已用) / 0.68GB(保留) / 1.69GB(峰值)
💻 系统内存: 14.73GB(已用) / 15.81GB(总计) / 93.1%
------------------------------------------------------------
